In [163]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
import glob as gl
import os

In [164]:
path = "src/results"

""" datetime = "/2011-03-24_15"
df = pd.read_csv(path + datetime + '.csv')  """


dfs = []

for file in gl.glob(path + '/20*.csv'):
    dfs.append(pd.read_csv(file))

df = pd.concat(dfs, ignore_index=True)
df.count() #cada csv tiene 160920 filas

overflow                            482760
cap                                 482760
load                                482760
dist                                482760
origen_id                           482760
dest_id                             482760
len_origen_tag                      482760
len_dest_tag                        482760
modelo                              482760
criterion                           482760
degree                              482760
total_balance                       482760
abs_flux                            482760
timestamp                           482760
datetime                            482760
h                                   482760
Beam Irradiance (W/m2)              482760
Diffuse Irradiance (W/m2)           482760
Ambient Temperature (C)             482760
Plane of Array Irradiance (W/m2)    482760
Cell Temperature (C)                482760
DC Array Output (W)                 482760
Pavg                                482760
dif        

## Encoder

In [165]:
modelo = df.iloc[:, 8].values 
from sklearn.preprocessing import LabelEncoder
modelo = LabelEncoder().fit_transform(modelo) #codificación del modelo

In [166]:
df = df.drop(df.columns[8], axis=1) #se elimina la antigua con los strings del modelo
df['modelo'] = modelo #se añade la nueva codificada al final

X = df.iloc[:, 1:] 
X = X.drop(['datetime', 'timestamp', 'load', 'DC Array Output (W)' , 'Pavg', 'dif'], axis=1)
y = df.iloc[:, 0].values #valores de overflow

X.columns

Index(['cap', 'dist', 'origen_id', 'dest_id', 'len_origen_tag', 'len_dest_tag',
       'criterion', 'degree', 'total_balance', 'abs_flux', 'h',
       'Beam Irradiance (W/m2)', 'Diffuse Irradiance (W/m2)',
       'Ambient Temperature (C)', 'Plane of Array Irradiance (W/m2)',
       'Cell Temperature (C)', 'modelo'],
      dtype='object')

## ANN Model

In [167]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [168]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [169]:
model = keras.Sequential([
    
    #Adding the input layer and the first hidden layer
    #se prueba con 6, los valores se obtienen probando o con alguna técnica como k Fold Cross Validation
    keras.layers.Dense(6, input_shape=(X.shape[1],), activation='relu'), #nº de entradas
    
    #Adding the second hidden layer
    #mismo valor que arriba
    keras.layers.Dense(6, activation='relu'),
    
    #Adding the output layer -> 1 salida: 0 o 1
    #*si no es salida binaria, se indica el número de posibles salidas y activation='softmax'
    keras.layers.Dense(1, activation='sigmoid')
])

#model.summary() #resumen de la estructura de la red neuronal
#los parámetros son los pesos

In [170]:
#binary_crossentropy -> clasificación binaria (si/no)
#*si no es salida binaria, se aplica categorical_crossentropy o sparse_categorical_crossentropy
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [171]:
model.fit(X_train, y_train, batch_size = 50, epochs = 8)

Epoch 1/8
7725/7725 [==============================] - 15s 2ms/step - loss: 0.0915 - accuracy: 0.9746
Epoch 2/8
7725/7725 [==============================] - 14s 2ms/step - loss: 0.0676 - accuracy: 0.9805
Epoch 3/8
7725/7725 [==============================] - 14s 2ms/step - loss: 0.0631 - accuracy: 0.9810
Epoch 4/8
7725/7725 [==============================] - 14s 2ms/step - loss: 0.0607 - accuracy: 0.9814
Epoch 5/8
7725/7725 [==============================] - 16s 2ms/step - loss: 0.0603 - accuracy: 0.9816
Epoch 6/8
7725/7725 [==============================] - 16s 2ms/step - loss: 0.0602 - accuracy: 0.9815
Epoch 7/8
7725/7725 [==============================] - 17s 2ms/step - loss: 0.0601 - accuracy: 0.9816
Epoch 8/8
7725/7725 [==============================] - 18s 2ms/step - loss: 0.0601 - accuracy: 0.9817


In [172]:
#se obtienen las pérdidas y la precisión del modelo
test_loss, test_accuracy = model.evaluate(X_test, y_test)
#si la precisión obtenida aquí es mucho menor que la de las épocas, se está sobreentrenando el modelo

3018/3018 [==============================] - 7s 2ms/step - loss: 0.0592 - accuracy: 0.9821


In [173]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)

#comparación entre las predicciones y el test
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

3018/3018 [==============================] - 5s 2ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [174]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[94563   122]
 [ 1607   260]]


0.9820925511641395